In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import requests
import json
import time
import plotly.express as px
import plotly.graph_objects as go
import copy
from tqdm.notebook import tqdm
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

In [2]:
def get_radiation_data(API_token, start_year, end_year, lat, long, capacity, loss, track, tilt, azimuth):

    token = API_token
    api_base = 'https://www.renewables.ninja/api/'

    s = requests.session()
    s.headers = {'Authorization': 'Token ' + token} # Send token header with each request
    url = api_base + 'data/pv'

    ###### PV data request
    DATA = pd.DataFrame()
    for year in tqdm(range(start_year, end_year+1)):

        args = {
            'lat': lat,
            'lon': long,
            'date_from': f'{year}-01-01',
            'date_to': f'{year}-12-31',
            'dataset': 'merra2',
            'capacity': capacity,
            'system_loss': loss,
            'tracking': track,    # 0: fixed, 1: single axis (azimuth), 2: double axis (azimuth and tilt)
            'tilt': tilt,         # should be equal to latitude value for good annual performance
            'azim': azimuth,
            'format': 'json',
            'raw': True,
            'header': False
        }

        r = s.get(url, params=args)

        # Parse JSON to get a pandas.DataFrame of data and dict of metadata
        parsed_response = json.loads(r.text)

        data = pd.read_json(json.dumps(parsed_response), orient='index')
        data[f'total_irradiance_{year}'] = data['irradiance_direct'] + data['irradiance_diffuse'] # kWh/m2
        data[f'direct_{year}'] = data['irradiance_direct'] # kWh/m2
        data[f'diffuse_{year}'] = data['irradiance_diffuse'] # kWh/m2
        data[f'ele_{year}'] = data['electricity'] # kWh
        data[f'temp_{year}'] = data['temperature'] # C
        
        data = data.drop(columns=['electricity','irradiance_direct','irradiance_diffuse','temperature'])

        data = data.reset_index(drop=True)

        DATA = pd.concat([DATA, data], axis=1)

        time.sleep(3)

    if len(DATA) > 8760:
        DATA = DATA.iloc[:8760]
    
    DATA.index = pd.date_range(start="2023-01-01 00:00", periods=8760, freq="H") #2023 is just a sample year for df index
    
    return DATA

In [3]:
API_token = 'renewable ninja token here'
start_year = 2000
end_year = 2023
lat = 35.4215
long = 51.4881
capacity = 1   # kW
loss = 0       # [0,1]
track = 0      # 0: fixed, 1: single axis (azimuth), 2: double axis (azimuth and tilt)
tilt = 35      # degree (should be equal to latitude value for good annual performance)
azimuth = 180  # south facing

In [4]:
# columns = [total_irradiance_year, direct_year, diffuse_year, ele_year, temp_year]
data = get_radiation_data(API_token, start_year, end_year, lat, long, capacity, loss, track, tilt, azimuth)

  0%|          | 0/24 [00:00<?, ?it/s]

In [4]:
#data.to_excel('solar_data.xlsx') # save the data in excel to prevent running the above code again to get the data
data = pd.read_excel('solar_data.xlsx', header=0, index_col=0)

In [5]:
years = list(range(start_year, end_year +1))
columns = ['total_irradiance', 'direct', 'diffuse', 'ele', 'temp']

In [6]:
for col in columns:
    temp_mean = data.filter(like=col).mean(axis=1)
    temp_std = data.filter(like=col).std(axis=1)
    temp_min = data.filter(like=col).min(axis=1)
    temp_max = data.filter(like=col).max(axis=1)
    
    data[col+'_mean'] = temp_mean
    data[col+'_std'] = temp_std
    data[col+'_min'] = temp_min
    data[col+'_max'] = temp_max

In [8]:
# Define month dropdown options and mapping.
month_options = [{"label": m, "value": m} for m in 
                 ["All", "January", "February", "March", "April", "May", "June", 
                  "July", "August", "September", "October", "November", "December"]]
month_to_num = {
    "January": 1, "February": 2, "March": 3, "April": 4, "May": 5, "June": 6,
    "July": 7, "August": 8, "September": 9, "October": 10, "November": 11, "December": 12
}

# Extend the year-dropdown options to include the aggregated options.
year_options = [{"label": str(y), "value": y} for y in years]
year_options.append({"label": "Mean with 95% CI", "value": "mean"})
year_options.append({"label": "Min/Max", "value": "minmax"})

# Initialize the JupyterDash app.
app = JupyterDash(__name__)

# Layout: Three dropdowns (Year, Feature, Month) arranged side by side.
app.layout = html.Div([
    html.Div([
        html.Div([
            html.Label("Year(s):"),
            dcc.Dropdown(
                id="year-dropdown",
                options=year_options,
                value=[2023],
                multi=True,
                clearable=False
            )
        ], style={"flex": "1", "padding": "10px"}),

        html.Div([
            html.Label("Feature:"),
            dcc.Dropdown(
                id="feature-dropdown",
                options=[{"label": col, "value": col} for col in columns],
                value="temp",
                clearable=False
            )
        ], style={"flex": "1", "padding": "10px"}),

        html.Div([
            html.Label("Month:"),
            dcc.Dropdown(
                id="month-dropdown",
                options=month_options,
                value="All",
                clearable=False
            )
        ], style={"flex": "1", "padding": "10px"}),

        html.Div([
            html.Label("Week (when month is not all):"),
            dcc.Dropdown(
                id="week-dropdown",
                options=[{"label": "All", "value": "all"}] +  
                        [{"label": f"Week {i}", "value": i} for i in range(1, 5)] +
                        [{"label": "Rest", "value": "rest"}],
                value="all",
                clearable=False
            )
        ], style={"flex": "1", "padding": "10px"}),
    
    ], style={"display": "flex", "flexDirection": "row", "justifyContent": "center"}),
    
    html.Div([
        html.Label("Date Range (when month is all, just month and day are important):"),
        dcc.DatePickerRange(
            id="date-picker-range",
            start_date=None,  # Default to None (no filter)
            end_date=None,
            display_format="YYYY-MM-DD",  # User-friendly format
            clearable=True
        )
    ], style={"flex": "1", "padding": "10px"}),

    dcc.Graph(id="time-series-plot")
])

# Callback: Update the plot based on selected years, feature, and month.
@app.callback(
    Output("time-series-plot", "figure"),
    [Input("year-dropdown", "value"),
     Input("feature-dropdown", "value"),
     Input("month-dropdown", "value"),
     Input("week-dropdown", "value"),
     Input("date-picker-range", "start_date"),  # New input
     Input("date-picker-range", "end_date")]  # New input
)
def update_plot(selected_years, feature, selected_month, selected_week, start_date, end_date):
    
    # Ensure selected_years is a list.
    if not isinstance(selected_years, list):
        selected_years = [selected_years]
    
    # Check if an aggregated option is selected.
    agg_mode = None
    for val in selected_years:
        if isinstance(val, str):
            agg_mode = val
            break

    # Apply month filtering
    if selected_month != "All":
        month_num = month_to_num[selected_month]
        if selected_week != "all":
            if selected_week in range(1, 5):  # Weeks 1-4 (Days 1-28):
                mask =(data.index.month==month_num)&(data.index.day>(selected_week-1)*7)&(data.index.day<=selected_week*7)
            elif selected_week == "rest":  # Rest of the month (Days > 28):
                mask = (data.index.month == month_num)&(data.index.day>28)
        else:
            mask = (data.index.month == month_num)  # Filter rows for the selected month
    else:
        if start_date and end_date:
            start_date = pd.to_datetime(start_date)  # Convert to datetime
            end_date = pd.to_datetime(end_date)

            # Extract only month and day from the index
            month_day = data.index.strftime("%m-%d")  # Format index as "MM-DD"

            # Extract only month and day from selected start/end dates
            start_md = start_date.strftime("%m-%d")
            end_md = end_date.strftime("%m-%d")

            # Apply mask based on month and day, ignoring the year
            mask = (month_day >= start_md) & (month_day <= end_md)
        else:
            mask = slice(None)  # If "All", use the entire data
    
    x_vals = data.index[mask]  # Final x-values after applying both filters

    fig = go.Figure()

    if "mean" in selected_years or "minmax" in selected_years:
        agg_mode = "mean" if "mean" in selected_years else "minmax"
        selected_years = [y for y in selected_years if isinstance(y, int)]  # Keep only numeric years

        
        if agg_mode == "mean":
            
            mean_series = data[f"{feature}_mean"][mask]
            std_series = data[f"{feature}_std"][mask]
            
            # Set std to zero when mean is zero
            # if mean_series != 0 is flase (so mean =0), then set std to 0
            std_series = std_series.where(mean_series != 0, 0)
            
            # Compute the upper and lower bounds
            upper_bound = mean_series + 1.96 * std_series
            lower_bound = mean_series - 1.96 * std_series
            if feature in ['total_irradiance', 'direct', 'diffuse']:
                lower_bound = lower_bound.clip(lower=0)

            # Confidence interval fill
            fig.add_trace(go.Scatter(
                x=list(x_vals),
                y=list(upper_bound),
                mode='lines',
                line=dict(width=0),
                showlegend=False,
                hoverinfo='skip'
            ))
            fig.add_trace(go.Scatter(
                x=list(x_vals),
                y=list(lower_bound),
                mode='lines',
                fill='tonexty',
                fillcolor='rgba(0,100,80,0.5)',
                line=dict(width=0),
                name='95% CI',
                hoverinfo='skip'
            ))
            # Mean line
            fig.add_trace(go.Scatter(
                x=list(x_vals),
                y=list(mean_series),
                mode='lines',
                line=dict(color='rgb(0,100,80)'),
                name="Mean"
            ))

        elif agg_mode == "minmax":
            min_series = data[f"{feature}_min"][mask]
            max_series = data[f"{feature}_max"][mask]

            fig.add_trace(go.Scatter(
                x=list(x_vals),
                y=list(min_series),
                mode='lines',
                name="Min",
                line=dict(color='lightblue')
            ))
            fig.add_trace(go.Scatter(
                x=list(x_vals),
                y=list(max_series),
                mode='lines',
                name="Max",
                line=dict(color='lightblue'),
                fill='tonexty'
            ))

    # Plot each selected numeric year
    for yr in selected_years:
        col_name = f"{feature}_{yr}"
        series = data[col_name][mask]
        trace_name = f"{yr} - {selected_month}" if selected_month != "All" else str(yr)

        fig.add_trace(go.Scatter(
            x=list(x_vals),
            y=list(series),
            mode='lines',
            name=trace_name
        ))

    # Update title
    title = f"Yearly Data for {', '.join(map(str, selected_years))} - {feature.capitalize()}"
    if "mean" in selected_years:
        title += " + Mean with 95% CI"
    if "minmax" in selected_years:
        title += " + Min/Max"

    fig.update_layout(
        title=title,
        xaxis_title="Time",
        yaxis_title=feature.capitalize(),
        template="plotly_dark"
    )
    return fig

# Run the app inside Jupyter Notebook.
app.run_server(debug=True, use_reloader=False, port=8050)

Dash app running on http://127.0.0.1:8051/


In [ ]:
temp_mean = data.mean(axis=1)
temp_max = data.max(axis=1)
temp_min = data.min(axis=1)
temp_std = data.std(axis=1)

data['mean'] = temp_mean
data['max'] = temp_max
data['min'] = temp_min
data['std'] = temp_std

In [ ]:
px.line(data, y='mean', title="Mean Solar Radiation (Wh/m2)")

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=data['max'], mode='lines', name='Max'))
fig.add_trace(go.Scatter(y=data['min'], mode='lines', name='Min'))

# plot without range selector
# fig.update_layout(title="Max & Min Plot", xaxis_title="Index", yaxis_title="Value")

# plot with range selector
fig.update_layout(
    title="Hourly Min & MAX (Wh/m2)",
    xaxis_title="Index",
    yaxis_title="Value",
    xaxis=dict(
        rangeslider=dict(visible=True),  # Show the range slider below the plot
        type="linear",  # Type of x-axis
        tickmode="array",  # Set x-axis ticks
        tickvals=list(range(0, len(data), 168)),  # Set weekly x-axis ticks at intervals
        ticktext=[f'{i}' for i in range(0, len(data), 168)]  # Display x-axis ticks as 0, 168, 336...
    )
)

fig.show()

In [ ]:
data['upper'] = data['mean'] + 2 * data['std']
data['lower'] = data['mean'] - 2 * data['std']
data['lower'] = data['lower'].clip(lower=0)

fig = go.Figure()
fig.add_trace(go.Scatter(y=data['mean'], mode='lines', name='Mean', line=dict(color='blue')))
fig.add_trace(go.Scatter(y=data['upper'], mode='lines', name='Upper Bound (+2 std)', line=dict(color='red', width=0.5)))
fig.add_trace(go.Scatter(y=data['lower'], mode='lines', name='Lower Bound (-2 std)', line=dict(color='red', width=0.5)))

# Add shaded area between upper and lower bound (±2 std range)
fig.add_trace(go.Scatter(
    x=data.index, y=data['upper'], fill='tonexty', fillcolor='rgba(255, 0, 0, 0.2)',
    line=dict(color='rgba(255,255,255,0)'), name='2 Std Range', showlegend=False))

# plot without range selector
# fig.update_layout(title="Mean with ±2 Std Deviation Range", xaxis_title="Index", yaxis_title="Value")
# plot with range selector
fig.update_layout(
    title="Hourly Mean with ±2 Std Deviation Range (Wh/m2)",
    xaxis_title="Index",
    yaxis_title="Value",
    xaxis=dict(
        rangeslider=dict(visible=True),  # Show the range slider below the plot
        type="linear",  # Type of x-axis
        tickmode="array",  # Set x-axis ticks
        tickvals=list(range(0, len(data), 168)),  # Set x-axis ticks at intervals
        ticktext=[f'{i}' for i in range(0, len(data), 168)]  # Display x-axis ticks as 0, 24, 48...
    )
)

fig.show()

In [ ]:
plt.figure(figsize=(8, 4))
plt.hist(data[data['mean']>0]['mean'], edgecolor='black', color='cyan', density=False, bins = 'fd');
plt.ylabel('Frequency')
plt.xlabel('Mean Solar Radiation (Wh/m2)')
plt.title('Distribution of Mean hourly Solar Radiation')

In [ ]:
plt.figure(figsize=(8, 4))
plt.bar(data.sum(axis=0).iloc[:-6].index, data.sum(axis=0).iloc[:-6].values/1000, edgecolor='black', color='orange') # /1000 to chnage Wh to kWh
plt.ylabel('kWh/m2')
plt.xlabel('Year')
plt.xticks(rotation=90)
plt.title('Total Solar Radiation per Year')

In [ ]:
data = data.iloc[:, :-6] # remove mean, std, max, min, upper(mean+2std), lower(mean-2std)
data['mean_prev_year'] = data.iloc[:,:-1].mean(axis=1) # claculate mean radiation in previous years to be used as an input feature
data = data.iloc[:,-2:] # remove previous years data

In [ ]:
# add lagged features to the dataset
for lag in range(1, 4):
    data[f'total_irradiance_{end_year}_lag_{lag}'] = data[f'total_irradiance_{end_year}'].shift(lag)
data.dropna(inplace=True) # drop rows with NaN values created as a result of lag features

In [ ]:
# move the target variable (total_irrandiance_{end_year}) to the end of the dataframe
target = data.pop(f'total_irradiance_{end_year}')
data[f'total_irradiance_{end_year}'] = target

In [ ]:
train_percentage = 0.8 # percentage of data in the train set
cutpoint = int(train_percentage*len(data)) # find the cutpoint to separate train and test data

train = data[:cutpoint]
test = data[cutpoint:]

y_train, X_train = train.iloc[:,-1].values, train.iloc[:,:-1].values
y_test, X_test = test.iloc[:,-1].values, test.iloc[:,:-1].values

In [ ]:
MMS = MinMaxScaler()
X_train = MMS.fit_transform(X_train)
X_test = MMS.transform(X_test)

In [ ]:
model = MLPRegressor((80,80,), solver='lbfgs', random_state=1)

In [ ]:
model.fit(X_train, y_train);

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# integrating domain knowledge to prevent having negative values right at the
# start of the night
y_train_pred = np.where(X_train[:,0] == 0, 0, y_train_pred)
y_test_pred = np.where(X_test[:,0] == 0, 0, y_test_pred)

rmse_train = RMSE(y_train, y_train_pred)
rmse_test = RMSE(y_test, y_test_pred)

print('RMSE on train data: ', rmse_train)
print('RMSE on test data: ', rmse_test)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=y_test, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(y=y_test_pred, mode='lines', name='Predicted'))

# plot with range selector
fig.update_layout(
    title="Test performance",
    xaxis_title="Index",
    yaxis_title="Value",
    xaxis=dict(
        rangeslider=dict(visible=True),  # Show the range slider below the plot
        type="linear",  # Type of x-axis
        tickmode="array",  # Set x-axis ticks
        tickvals=list(range(0, len(y_test), 24)),  # Set weekly x-axis ticks at intervals
        ticktext=[f'{i}' for i in range(0, len(y_test), 24)]  # Display x-axis ticks as 0, 168, 336...
    )
)

fig.show()

In [ ]:
plt.figure(figsize=(10,4), dpi=300)
plt.scatter(y_test, y_test_pred, edgecolor='black', color='yellow')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Test')

In [ ]:
plt.figure(figsize=(10,4), dpi=300)
plt.scatter(y_train, y_train_pred, edgecolor='black', color='yellow')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Train')

In [ ]:
# series multistep prediction
# the model uses its own predictions to make new predictions

test_copy = copy.deepcopy(test)

update_every = 6 # every 6 hours use the real data istead of models predictions to make new predictions
predictions = []
for i in range(len(test_copy)):

    if test_copy.iloc[i,0] == 0: # column 0 is the mean value, if its zero it means that it is night
        prediction = 0
    else:
        xTest = MMS.transform(test_copy.iloc[i,:-1].values.reshape(1, -1)) # omit the last column (target) and transform
        prediction = model.predict(xTest)[0]

    predictions.append(prediction)

    if i != len(test_copy) -1:
        if i % update_every != 0:
            test_copy.iloc[i+1,1] = prediction           # column 1 is the first lag which takes the predicted value of the previous time step
            test_copy.iloc[i+1,2] = test_copy.iloc[i,1]  # column 2 is the second lag which takes the value of first lag of the previous time step
            test_copy.iloc[i+1,3] = test_copy.iloc[i,2]  # column 3 is the third lag which takes the value of second lag of the previous time step

In [ ]:
rmse_test_multistep = RMSE(y_test, predictions)
print('RMSE on test data (multistep): ', rmse_test_multistep)
print(f'CVRMSE = {rmse_test_multistep/y_test[np.where(y_test>0)].mean()}')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=y_test, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(y=predictions, mode='lines', name='6 hours predictions'))

# plot with range selector
fig.update_layout(
    title="Test performance",
    xaxis_title="Index",
    yaxis_title="Value",
    xaxis=dict(
        rangeslider=dict(visible=True),  # Show the range slider below the plot
        type="linear",  # Type of x-axis
        tickmode="array",  # Set x-axis ticks
        tickvals=list(range(0, len(y_test), 24)),  # Set weekly x-axis ticks at intervals
        ticktext=[f'{i}' for i in range(0, len(y_test), 24)]  # Display x-axis ticks as 0, 168, 336...
    )
)

fig.show()

In [ ]:
# parallel multi step
data_copy = copy.deepcopy(data)
# creating the datasets
for lag in range(-1,-6,-1):
    data_copy[f'total_irradiance_2023_next_{-lag}'] = data_copy['total_irradiance_2023'].shift(lag)
    data_copy[f'mean_prev_year_next_{-lag}'] = data_copy['mean_prev_year'].shift(lag)
data_copy.dropna(inplace=True)

In [ ]:
train_percentage = 0.8
cutpoint = int(train_percentage*len(data_copy))

#################

data_now = data_copy[['mean_prev_year','total_irradiance_2023_lag_1','total_irradiance_2023_lag_2',
                      'total_irradiance_2023_lag_3','total_irradiance_2023']]
train = data_now[:cutpoint]
test = data_now[cutpoint:]

y_train_now, X_train_now = train.iloc[:,-1].values, train.iloc[:,:-1].values
y_test_now, X_test_now = test.iloc[:,-1].values, test.iloc[:,:-1].values

MMS_now = MinMaxScaler()
X_train_now = MMS_now.fit_transform(X_train_now)
X_test_now = MMS_now.transform(X_test_now)

#################

data_next_1 = data_copy[['mean_prev_year_next_1','total_irradiance_2023_lag_1','total_irradiance_2023_lag_2',
                         'total_irradiance_2023_lag_3','total_irradiance_2023_next_1']]
train = data_next_1[:cutpoint]
test = data_next_1[cutpoint:]

y_train_next_1, X_train_next_1 = train.iloc[:,-1].values, train.iloc[:,:-1].values
y_test_next_1, X_test_next_1 = test.iloc[:,-1].values, test.iloc[:,:-1].values

MMS_next_1 = MinMaxScaler()
X_train_next_1 = MMS_next_1.fit_transform(X_train_next_1)
X_test_next_1 = MMS_next_1.transform(X_test_next_1)

#################

data_next_2 = data_copy[['mean_prev_year_next_2','total_irradiance_2023_lag_1','total_irradiance_2023_lag_2',
                         'total_irradiance_2023_lag_3','total_irradiance_2023_next_2']]
train = data_next_2[:cutpoint]
test = data_next_2[cutpoint:]

y_train_next_2, X_train_next_2 = train.iloc[:,-1].values, train.iloc[:,:-1].values
y_test_next_2, X_test_next_2 = test.iloc[:,-1].values, test.iloc[:,:-1].values

MMS_next_2 = MinMaxScaler()
X_train_next_2 = MMS_next_2.fit_transform(X_train_next_2)
X_test_next_2 = MMS_next_2.transform(X_test_next_2)

#################
data_next_3 = data_copy[['mean_prev_year_next_3','total_irradiance_2023_lag_1','total_irradiance_2023_lag_2',
                         'total_irradiance_2023_lag_3','total_irradiance_2023_next_3']]
train = data_next_3[:cutpoint]
test = data_next_3[cutpoint:]

y_train_next_3, X_train_next_3 = train.iloc[:,-1].values, train.iloc[:,:-1].values
y_test_next_3, X_test_next_3 = test.iloc[:,-1].values, test.iloc[:,:-1].values

MMS_next_3 = MinMaxScaler()
X_train_next_3 = MMS_next_3.fit_transform(X_train_next_3)
X_test_next_3 = MMS_next_3.transform(X_test_next_3)

#################
data_next_4 = data_copy[['mean_prev_year_next_4','total_irradiance_2023_lag_1','total_irradiance_2023_lag_2',
                         'total_irradiance_2023_lag_3','total_irradiance_2023_next_4']]
train = data_next_4[:cutpoint]
test = data_next_4[cutpoint:]

y_train_next_4, X_train_next_4 = train.iloc[:,-1].values, train.iloc[:,:-1].values
y_test_next_4, X_test_next_4 = test.iloc[:,-1].values, test.iloc[:,:-1].values

MMS_next_4 = MinMaxScaler()
X_train_next_4 = MMS_next_4.fit_transform(X_train_next_4)
X_test_next_4 = MMS_next_4.transform(X_test_next_4)

#################
data_next_5 = data_copy[['mean_prev_year_next_5','total_irradiance_2023_lag_1','total_irradiance_2023_lag_2',
                         'total_irradiance_2023_lag_3','total_irradiance_2023_next_5']]
train = data_next_5[:cutpoint]
test = data_next_5[cutpoint:]

y_train_next_5, X_train_next_5 = train.iloc[:,-1].values, train.iloc[:,:-1].values
y_test_next_5, X_test_next_5 = test.iloc[:,-1].values, test.iloc[:,:-1].values

MMS_next_5 = MinMaxScaler()
X_train_next_5 = MMS_next_5.fit_transform(X_train_next_5)
X_test_next_5 = MMS_next_5.transform(X_test_next_5)

In [ ]:
model_now = MLPRegressor((80,80,), solver='lbfgs', random_state=1)
model_now.fit(X_train_now, y_train_now);

model_next_1 = MLPRegressor((80,80,), solver='lbfgs', random_state=1)
model_next_1.fit(X_train_next_1, y_train_next_1);

model_next_2 = MLPRegressor((80,80,), solver='lbfgs', random_state=1)
model_next_2.fit(X_train_next_2, y_train_next_2);

model_next_3 = MLPRegressor((80,80,), solver='lbfgs', random_state=1)
model_next_3.fit(X_train_next_3, y_train_next_3);

model_next_4 = MLPRegressor((80,80,), solver='lbfgs', random_state=1)
model_next_4.fit(X_train_next_4, y_train_next_4);

model_next_5 = MLPRegressor((80,80,), solver='lbfgs', random_state=1)
model_next_5.fit(X_train_next_5, y_train_next_5);

In [ ]:
y_test_pred_now = model_now.predict(X_test_now)
y_test_pred_now = np.where(X_test_now[:,0] == 0, 0, y_test_pred_now)
rmse_test_now = RMSE(y_test_now, y_test_pred_now)

y_test_pred_next_1 = model_next_1.predict(X_test_next_1)
y_test_pred_next_1 = np.where(X_test_next_1[:,0] == 0, 0, y_test_pred_next_1)
rmse_test_next_1 = RMSE(y_test_next_1, y_test_pred_next_1)

y_test_pred_next_2 = model_next_2.predict(X_test_next_2)
y_test_pred_next_2 = np.where(X_test_next_2[:,0] == 0, 0, y_test_pred_next_2)
rmse_test_next_2 = RMSE(y_test_next_2, y_test_pred_next_2)

y_test_pred_next_3 = model_next_3.predict(X_test_next_3)
y_test_pred_next_3 = np.where(X_test_next_3[:,0] == 0, 0, y_test_pred_next_3)
rmse_test_next_3 = RMSE(y_test_next_3, y_test_pred_next_3)

y_test_pred_next_4 = model_next_4.predict(X_test_next_4)
y_test_pred_next_4 = np.where(X_test_next_4[:,0] == 0, 0, y_test_pred_next_4)
rmse_test_next_4 = RMSE(y_test_next_4, y_test_pred_next_4)

y_test_pred_next_5 = model_next_5.predict(X_test_next_5)
y_test_pred_next_5 = np.where(X_test_next_5[:,0] == 0, 0, y_test_pred_next_5)
rmse_test_next_5 = RMSE(y_test_next_5, y_test_pred_next_5)

print('RMSE on test now: ', rmse_test_now)
print('RMSE on test next 1: ', rmse_test_next_1)
print('RMSE on test next 2: ', rmse_test_next_2)
print('RMSE on test next 3: ', rmse_test_next_3)
print('RMSE on test next 4: ', rmse_test_next_4)
print('RMSE on test next 5: ', rmse_test_next_5)

In [ ]:
X_test_now[0]

In [ ]:
predictions = []
for i in range(0,len(X_test_now),6):
    predictions.append(model_now.predict(X_test_now[i].reshape(1, -1))[0] if X_test_now[i,0]!=0 else 0)
    predictions.append(model_next_1.predict(X_test_next_1[i].reshape(1, -1))[0] if X_test_next_1[i,0]!=0 else 0)
    predictions.append(model_next_2.predict(X_test_next_2[i].reshape(1, -1))[0] if X_test_next_2[i,0]!=0 else 0)
    predictions.append(model_next_3.predict(X_test_next_3[i].reshape(1, -1))[0] if X_test_next_3[i,0]!=0 else 0)
    predictions.append(model_next_4.predict(X_test_next_4[i].reshape(1, -1))[0] if X_test_next_4[i,0]!=0 else 0)
    predictions.append(model_next_5.predict(X_test_next_5[i].reshape(1, -1))[0] if X_test_next_5[i,0]!=0 else 0)

predictions.pop()

In [ ]:
RMSE(y_test_now,predictions)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=y_test_now, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(y=predictions, mode='lines', name='6 hours predictions'))

# plot with range selector
fig.update_layout(
    title="Test performance",
    xaxis_title="Index",
    yaxis_title="Value",
    xaxis=dict(
        rangeslider=dict(visible=True),  # Show the range slider below the plot
        type="linear",  # Type of x-axis
        tickmode="array",  # Set x-axis ticks
        tickvals=list(range(0, len(y_test), 24)),  # Set weekly x-axis ticks at intervals
        ticktext=[f'{i}' for i in range(0, len(y_test), 24)]  # Display x-axis ticks as 0, 168, 336...
    )
)

fig.show()